In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys
utils_path=os.path.join(os.getcwd(),'..','..')

sys.path.append(utils_path)

from utils.func import reuse


In [0]:
df_user=spark.readStream.format("cloudFiles").option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimUser/checkpoint").load("abfss://bronze@pratikazurestorage.dfs.core.windows.net/DimUser/DimUser_*")

In [0]:
display(df_user)

In [0]:
df_user=df_user.withColumn("user_name",upper(col("user_name")))
display(df_user)

In [0]:
display(df_user)

In [0]:
 reuse_obj=reuse()

 df_user=reuse_obj.dropCols(df_user,['_rescued_data'])
 df_user=df_user.dropDuplicates()

 display(df_user)



In [0]:
df_user.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimUser/checkpoint").trigger(once=True).option("path","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimUser/data").toTable("spotify_catalog.silver.DimUser")

In [0]:
df_art=spark.readStream.format("cloudFiles").option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimArtist/checkpoint").option("schemaEvolutionMode","addNewColumns").load("abfss://bronze@pratikazurestorage.dfs.core.windows.net/DimArtist/DimArtist_*")

In [0]:
display(df_art)

In [0]:
df_art.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimArtist/checkpoint").trigger(once=True).option("path","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimArtist/data").toTable("spotify_catalog.silver.DimArtist")

In [0]:
df_track=spark.readStream.format("cloudFiles").option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimTrack/checkpoint").option("schemaEvolutionMode","addNewColumns").load("abfss://bronze@pratikazurestorage.dfs.core.windows.net/DimTrack/DimTrack_*")


In [0]:
display(df_track)

In [0]:
df_track=df_track.withColumn("duration_sec_flag",when(col("duration_sec")<150,"low")\
                            .when(col("duration_sec")<300,"medium")\
                            .otherwise("high"))
display(df_track)

In [0]:
df_track=df_track.withColumn("track_name",regexp_replace(col("track_name"),"-"," "))
df_track=reuse().dropCols(df_track,['_rescued_data'])
display(df_track)

In [0]:
df_track.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimTrack/checkpoint").trigger(once=True).option("path","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimTrack/data").toTable("spotify_catalog.silver.DimTrack")

In [0]:
df_date=spark.readStream.format("cloudFiles").option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimDate/checkpoint").option("schemaEvolutionMode","addNewColumns").load("abfss://bronze@pratikazurestorage.dfs.core.windows.net/DimDate/DimDate_*")

df_date=reuse().dropCols(df_date,['_rescued_data'])

df_date.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimDate/checkpoint").trigger(once=True).option("path","abfss://silver@pratikazurestorage.dfs.core.windows.net/DimDate/data").toTable("spotify_catalog.silver.DimDate")

In [0]:
df_fact=spark.readStream.format("cloudFiles").option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/FactStream/checkpoint").option("schemaEvolutionMode","addNewColumns").load("abfss://bronze@pratikazurestorage.dfs.core.windows.net/FactStream/FactStream_*")

df_fact=reuse().dropCols(df_fact,['_rescued_data'])

df_fact.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@pratikazurestorage.dfs.core.windows.net/FactStream/checkpoint").trigger(once=True).option("path","abfss://silver@pratikazurestorage.dfs.core.windows.net/FactStream/data").toTable("spotify_catalog.silver.FactStream")

In [0]:
%sql
select * from spotify_catalog.gold.dimtrack_gold
where track_id in (46,5)